In [ ]:
%%capture
!gdown --id 1utqJceSXendYxyRjulmj83-Iub2IxJWB # Download dialects_df.csv
!pip install PyArabic

In [ ]:
import pandas as pd
import re 
import pyarabic.araby as araby

In [ ]:
dialects_raw = pd.read_csv('dialects_df.csv',encoding='UTF-8', engine='python')
pd.set_option('display.max_colwidth', 500)

In [ ]:
print(dialects_raw.shape)

(147734, 3)


In [ ]:
dialects_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147734 entries, 0 to 147733
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       147734 non-null  object
 1   text     147732 non-null  object
 2   dialect  147718 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [ ]:
dialects_raw.dialect.value_counts()

EG    57635
LY    36499
LB    27617
SD    14433
MA    11534
Name: dialect, dtype: int64

In [ ]:
dialects_raw.sample(20)

,id,text,dialect
85015,1152692828767698944,@tamerabuarab ايه ده ؟! ده مسرح عمليات حربية مش خريطة توضيحية سياحية ابدا!! 😏,EG
100678,937586972687831040,@b_doooooooo يجي ويحط عليكم يالي تنشكوا انت عارفهم يارب😂😂😂😂😂😂,EG
126562,817789139097624576,بكرا مسكر خطي #سكر_خطك,LB
77089,882280673100353536,@D_D_DD_RU_ll_00 دواسه تويتر فاكر ال ATM لجان\nمخه لاسع ابن الوارمه,EG
64688,1159920385296601088,@saadresk بس نفسنا نعرف من على القمه,EG
97783,1057339680591675392,يا ريت يسمعوا بس هما مش عاوزينيشوفوا نجاحنا \nhttps://t.co/OEfWgIhi0K https://t.co/LxzC6a9Bf6,EG
143013,1176981047407497216,@Adam_Fadl7 الناس الفي الارض ماكافيه كلامي ماضد الصبر ولا أستعجال\nلو انا في الارض بعمل بصمه بتظهر للناس كلها دا ماهضم لحق اللجان لكن في جوانب كتيره مامغطيه كتيرا جدا,SD
80256,1143714041514397568,@Mazen_1409 ملعون ابوك خنزير جيفه,EG
78407,921010682602184832,لا رايق الولا عنتر ده https://t.co/qivqMllgZP,EG
127195,1186912974642044928,"كل شوي بيوصلنا تسجيل صوتي حدا عم يتفلسف فيه عن المظاهرة "" اليوم الأحزاب خصوصي الحركة و الحزب بدن يبعتو مندسين ما حدا يطلع من الشوارع و الجيش بدو يفتح الطرقات غصب عنكن"" يا خيي ما انت بالتسجيل هيدا بينت إنك مندسّ🙃 \nحاج كل واحد عم يغني الموال لبراسه!",LB


In [ ]:
dialects_raw.isnull().sum()

id          0
text        2
dialect    16
dtype: int64

In [ ]:
dialects_raw['text'].duplicated().sum()

5

In [ ]:
dialects_modified = dialects_raw.dropna().copy()
#drop id column
dialects_modified=dialects_modified.drop('id',axis=1)
# drop duplicates 
dialects_modified=dialects_modified.drop_duplicates('text')
# check for duplicates
dialects_modified.duplicated("text").sum()

0

### Cleaning and Preprocessing

In [ ]:
def clean_text(text):
    #remove duplicate letter
    text=re.sub(r'(.)\1+', r'\1', text).strip()
    text=re.sub(r"(.)(ـ)+", r'\1', text).strip()
    #normalize text
    text=araby.normalize_alef(text)
    
    text=araby.normalize_hamza(text)
    text=araby.strip_tashkeel(text)
    text=araby.strip_diacritics(text)
    text=araby.strip_harakat(text)
    #get just arabic text
    text=re.sub(r'[u0600-u06FF]+', '', text).strip()
    text=re.sub(r'[a-z]+', '', text).strip()
    #removing symbols 
    text=' '.join(w for w in re.split(r"\W", text) if w)
    #remove letters
    text=' '.join(w for w in araby.tokenize(text) if len(w)>1)
    return text


clean_text("@Mrs_Gibberish 😅😅😅  يحسس تحسيه ما يجلدش مفيش قناعه بالغلط نهائيا لْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ http://t.co/A4gRePxtjU")


'يحس تحسيه ما يجلدش مفيش قناعه بالغلط نهاءيا لحمد لله رب العالمين'

In [ ]:
dialects_modified['text']=dialects_modified['text'].apply(clean_text)
dialects_modified.sample(20)

,text,dialect
100701,صح كدي لان الفهم ليس بيه اجبار,EG
49521,احمد انا شاب مصري بدرس في امريكا عندي اكتءاب من فترة كبيرة بروح لاخصاءي نفسي بانتظام الدنيا باظت شوية اكتر مءخ اعتقد الماريجوانا ليها دور مهم يتفاقم اكتءابك ما تشتريش دقايق من المتعة بساعات من العذاب طالما بتروح لدكتور,EG
117407,بيقولوانك قلبك حنون,LB
86691,لو ده المنطق ماكانش نجح ولا خد اي جايزه اساسا بطلوا بقي افوره,EG
19020,ام ال وام تردها الجديد,LY
130763,صار تاني اجحش واحد بالعالم,LB
37066,كلمهتقرقعبمخك انا افهم شنو يعني تقرقع اول بعدين اقولكم السالفة,MA
76239,مش مهم هنخرج فين الاهم مع مين اصحاب مولتو مثلا,EG
85819,الواد عواد ده من كتر ما هو واخد قلم فا نفسه هياخده,EG
23035,تبي منهم حاجة تفضل الاحد نسالهم في طريقي هم,LY


In [ ]:
dialects_raw['text'].duplicated().sum()

5

In [ ]:
dialects_modified=dialects_modified.drop_duplicates('text')

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x, train_y,test_y = train_test_split(dialects_modified,dialects_modified['dialect'], test_size=0.1, random_state=42)
train_x.to_csv('train.csv',index=False, encoding='UTF-8')
test_x.to_csv('test.csv',index=False, encoding='UTF-8')

In [ ]:
import shutil
import os
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
files = ["train.csv","test.csv"]
folder_dir = "/content/drive/MyDrive/Colab Notebooks/NLP/Final-Project/Data/"
for file in files:
    dst_path = folder_dir+file
    shutil.copy2(src=file,
                dst=dst_path)